# Проект для "Викишоп"

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Требуется обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

**Ход работы:**
1. Загрузка и подготовка данных
2. Обучение моделей
3. Вывод

**Описание данных:**
- Данные находятся в файле `/datasets/toxic_comments.csv`.
- Столбец `text` содержит текст комментария, столбец `toxic` - целевой признак.

**Цель:**
- Построить модель со значением метрики качества F1 не меньше 0.75.

# 1. Подготовка

## Загрузка и проверка данных

In [1]:
# Импорты, настройки
import pandas as pd
import nltk
import re
import spacy

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

C:\Users\chernyse\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Константы
SEED = 777 # в деле со случайностями важна удача

In [3]:
df = pd.read_csv('toxic_comments.csv')

In [4]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df.duplicated().sum()

0

In [7]:
df.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

**Промежуточный вывод:**

Дубликатов нет, данные выглядят корректно. 

В файле имеется столбец с порядковыми номерами записей. Удалим его, так как он неинформативен.

Также обнаружили, что целевой признак имеет дисбаланс. Следует учесть это при разбивке объектов по выборкам.

## Подготовка данных

### Лемматизация текстов

In [8]:
df = df.drop(['Unnamed: 0'], axis=1)

Лемматизируем текст. Предварительно создадим копию имеющегося датафрейма, который изменим.

In [9]:
data = df.copy()

Для лемматизации английских текстов воспользуемся средствами библиотеки spaCy.

Протестируем его пригодность на случайном английском тексте:

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
example = "A plague o' both your houses! They have made worms' meat of me: I have it, And soundly too: your houses!"

doc = nlp(example)

" ".join([token.lemma_ for token in doc])

"a plague o ' both your house ! they have make worm ' meat of I : I have it , and soundly too : your house !"

Результат приемлем. Напишем функцию для обработки имеющегося датасета, а заодно функцию для очистки текста от ненужных символов.

In [11]:
def to_lemma(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [12]:
def clear_text(text):
    subed = re.sub(r'[^a-zA-Z]', ' ', text)
    subed = subed.split()
    subed = " ".join(subed)
    return subed

Лемматизированный текст сохраним в отдельном столбце для последующего сравнения.

In [13]:
%%time

data['lemm_text'] = data['text'].progress_apply(to_lemma).apply(clear_text)


Wall time: 49min 9s


In [14]:
# Проверяем случайные строки, ищем несоответствия
data.sample(10)

,text,toxic,lemm_text
8491,Personally I think the boxes are a better form...,0,personally I think the box be a well format an...
89770,Then I guess you'll have to deal with her.,0,then I guess you will have to deal with she
123636,I'm extremely inclined to revert User:QuackGur...,0,I be extremely inclined to revert User QuackGu...
83925,"""\n\nWhat do you think this means?\nI wonder w...",0,what do you think this mean I wonder what this...
138867,"BTW, as you've now updated your views at the b...",0,btw as you ve now update your view at the bott...
76651,"You are wrong on all points. And, you may be g...",0,you be wrong on all point and you may be guilt...
3235,"""\n\nThanks. That's three of us now, and coun...",0,thank that be three of we now and count but yo...
63445,I KNOW IT'S ENGLAND BECAUSE IT SAYS UTC \n\nIN...,1,I know it be ENGLAND because it say UTC in FAC...
120797,All fine! \n\nI honestly only had to put one c...,0,all fine I honestly only have to put one comma...
74257,A message for you \n\nFuck you!\n\nGAYFullbust...,1,a message for you fuck you GAYFullbuster Put t...


Обнаружили, что заглавные буквы изменяются только если она первая в строке. Переведём все буквы в нижний регистр.

In [15]:
data.lemm_text = data.lemm_text.str.lower()
data.sample(3)

,text,toxic,lemm_text
126827,"Hello, I am a student at the school whos IP ad...",0,hello i be a student at the school who s ip ad...
126443,Comments about C.Fred==\n\nYour edits make no ...,0,comment about c fred your edit make no sense y...
4620,That is completely unencyclopedic content whic...,0,that be completely unencyclopedic content whic...


Результат приемлем. Столбец `text` удаляем.

In [16]:
data = data.drop('text', axis=1)

### Подготовка признаков, векторизация

Так как заказчику необходима работа с неизвестными текстами, для корректного построения моделей разделим заранее выборку на обучающую и тестовую.

In [17]:
target = data.toxic
features = data.drop(['toxic'], axis=1)

Разделим выборку, где тестовая выборка будет составлять 25%. Также применим стратификацию.

In [18]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, stratify=target, test_size=0.25, random_state=SEED)

In [19]:
target_train.value_counts(normalize=True)

0    0.898392
1    0.101608
Name: toxic, dtype: float64

In [20]:
target_test.value_counts(normalize=True)

0    0.898375
1    0.101625
Name: toxic, dtype: float64

Проверяем результат разделения:

In [21]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(119469, 1)
(39823, 1)
(119469,)
(39823,)


Создадим корпусы для каждой выборки. Так как наша цель - создать модели для английских текстов и исходные данные заявлены как написанные латиницей тексты, кодировку оставим имеющуюся.

In [22]:
#Корпус тренировочной выборки
corpus_train = features_train['lemm_text']

#Корпус тестовой выборки для будущих тестов
corpus_test = features_test['lemm_text']

Загрузим стоп-слова и ознакомимся с их списком.

In [23]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chernyse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

Вычислим TF-IDF.

In [24]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Так как нам неизвестны будущие тексты (в данном случае - тестовые), функцию fit() применим только для обучающей выборки.

In [25]:
features_train = count_tf_idf.fit_transform(corpus_train)
features_test = count_tf_idf.transform(corpus_test)

In [26]:
print(features_train.shape)
print(features_test.shape)

(119469, 132891)
(39823, 132891)


In [34]:
features_train_0, features_temp, target_train_0, target_temp = train_test_split(
    features, target, stratify=target, test_size=0.4, random_state=SEED)

In [35]:
features_test_0, features_valid_0, target_test_0, target_valid_0 = train_test_split(
    features_temp, target_temp, stratify=target_temp, test_size=0.5, random_state=SEED)

In [36]:
print(features_train_0.shape)
print(features_valid_0.shape)
print(features_test_0.shape)
print(target_train_0.shape)
print(target_valid_0.shape)
print(target_test_0.shape)

(95575, 1)
(31859, 1)
(31858, 1)
(95575,)
(31859,)
(31858,)


In [37]:
corpus_train_0 = features_train_0['lemm_text']
corpus_valid_0 = features_valid_0['lemm_text']
corpus_test_0 = features_test_0['lemm_text']

In [38]:
features_train_0 = count_tf_idf.fit_transform(corpus_train_0)
features_valid_0 = count_tf_idf.transform(corpus_valid_0)
features_test_0 = count_tf_idf.transform(corpus_test_0)

Перейдём к построению моделей.

За метрики качества возьмём F1-меру, а также построим матрицу ошибок для детального понимания поведения моделей и последующих рекомендаций заказчику. Подготовим функцию для вывода метрик качества.

In [39]:
def show_me_quality_please(targets, predictions):
    print("Точность предсказаний:", round(accuracy_score(targets, predictions), 3))
    print('-'*25)
    print("F1-мера", round(f1_score(targets, predictions), 3))
    print('-'*25)
    print("Матрица ошибок:\n", confusion_matrix(targets, predictions, normalize='true')) #т.к. будем сравнимать матрицы разного объёма, нормализуем для наглядности

# 2. Обучение

## Логистическая регрессия

In [40]:
model_lr = LogisticRegression(max_iter=300, class_weight='balanced', random_state=SEED, C=10)

In [41]:
%%time

model_lr.fit(features_train_0, target_train_0)

Wall time: 38.1 s


LogisticRegression(C=10, class_weight='balanced', max_iter=300,
                   random_state=777)

In [42]:
prediction_lr_valid = model_lr.predict(features_valid_0)

In [43]:
show_me_quality_please(target_valid_0, prediction_lr_valid)

Точность предсказаний: 0.949
-------------------------
F1-мера 0.762
-------------------------
Матрица ошибок:
 [[0.96488715 0.03511285]
 [0.1940068  0.8059932 ]]


F1-мера на валидационной выборке составляет 0.762. Сравним с другими моделями для последующего выбора для испытаний на тестовой выборке.

In [44]:
#Оставил, чтобы было понятно, что это тут было.

#prediction_lr_test = model_lr.predict(features_test)
#show_me_quality_please(target_test, prediction_lr_test)

F1-мера достаточна для решения задачи.

Матрица ошибок показывает, что на тестовой выборке в >4 раз увеличилось количество ложноотрицательных ответов. Можем сделать предположение: люди используют нецензурные выражения на эмоциях, когда не обращают внимание на грамматику, совершая ошибки и создавая новые ругательные слова и n-граммы. Поэтому такие выражения для алгоритма являются уникальными и неизвестными.

Рассмотрим модели, построенные с использованием других алгоритмов, обратив внимание на распределение значений в матрице ошибок:

## Случайный лес

Подберём параметры с использованием GridSearchCV.

In [45]:
#%%time

#model_rfc = RandomForestClassifier(random_state=SEED)
#params_rfc = {'n_estimators': [50, 100],
#             'max_depth': [100]}
#grid_rfc = GridSearchCV(model_rfc, params_rfc, cv=5, n_jobs=-1)
#grid_rfc.fit(features_train, target_train)
#print(grid_rfc.best_params_)

In [46]:
pipeline_rfc = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('rfc', RandomForestClassifier(random_state=SEED))
])

In [47]:
pipeline_rfc_params = {'rfc__n_estimators': [3, 5],
                      'rfc__max_depth': [100]}

grid_search = GridSearchCV(pipeline_rfc, pipeline_rfc_params, n_jobs=-1, verbose=1, scoring='f1')

grid_search.fit(features_train_0, target_train_0)

print('Лучшая F1-мера:', grid_search.best_score_)
print('Лучшие параметры:', grid_search.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


Лучшая F1-мера: 0.31818794609936213
Лучшие параметры: {'rfc__max_depth': 100, 'rfc__n_estimators': 3}


In [48]:
model_rfc_0 = RandomForestClassifier(random_state=SEED, n_estimators=3, max_depth=100)
model_rfc_0.fit(features_valid_0, target_valid_0)
prediction_rfc_valid_0 = model_rfc_0.predict(features_valid_0)
show_me_quality_please(target_valid_0, prediction_rfc_valid_0)

Точность предсказаний: 0.933
-------------------------
F1-мера 0.52
-------------------------
Матрица ошибок:
 [[0.9988121  0.0011879 ]
 [0.64535063 0.35464937]]


In [49]:
#Оставил, чтобы было понятно, что это тут было.

#prediction_rfc_test = model_rfc.predict(features_test)
#show_me_quality_please(target_test, prediction_rfc_test)

Качество модели случайного леса слишком низкое для какой-либо работы. Также эта модель занимает слишком много памяти, что делает её ещё более неэффективной.

На матрице ошибок видим, что значение ложноположительных ответов слишком велико, по сравнению с моделью логистической регрессии.

**UPD** F1-мера на валидационной выборке недостаточна для дальнейшей работы с этой моделью.

## CatBoostClassifier

In [50]:
#model_cbc = CatBoostClassifier(random_state=SEED)
#params_cbc = {'depth': [8],
#             'learning_rate': [0.3]}

#grid_cbc = model_cbc.grid_search(params_cbc, X=features_train, y=target_train)
#grid_cbc['params']

In [51]:
#Время исполнения блока - 24 минуты

#model_cbc = CatBoostClassifier(random_state=SEED, iterations=100, depth=8, custom_metric='F1')
#model_cbc.fit(features_train, target_train)
#prediction_cbc_train = model_cbc.predict(features_train)
#show_me_quality_please(target_train, prediction_cbc_train)

*Копирую вывод предыдущего блока*

Точность предсказаний: 0.965

F1-мера 0.801

Матрица ошибок:

 [[0.99460542 0.00539458]
 
 [0.30068375 0.69931625]]

F1-мера достигает 0.8. Протестируем модель на тестовой выборке:

In [52]:
# Закомментировал, чтобы не выводилась ошибка

#prediction_cbc_test = model_cbc.predict(features_test)
#show_me_quality_please(target_test, prediction_cbc_test)

*Копирую вывод предыдущего блока*

Точность предсказаний: 0.956

F1-мера 0.752

Матрица ошибок:

[[0.9910275  0.0089725 ]

[0.34964171 0.65035829]]

Модель, построенная по алгоритмам CatBoost, имеет слишком долгое время обучения, не отражая это в повышенном качестве. Несмотря на то, что на тестовой выборке модель достигла F1-меры 0.752, количество ложноположительных ответов достигает 30%, что недопустимо для качественной работы и хуже модели логистической регрессии. 

In [53]:
#Время исполнения - около 40 минут.

model_cbc_0 = CatBoostClassifier(random_state=SEED, iterations=200, depth=8, custom_metric='F1')
model_cbc_0.fit(features_train_0, target_train_0)
prediction_cbc_valid_0 = model_cbc_0.predict(features_valid_0)
show_me_quality_please(target_valid_0, prediction_cbc_valid_0)

Learning rate set to 0.315861
0:	learn: 0.4272870	total: 23.7s	remaining: 1h 18m 36s
1:	learn: 0.3157364	total: 45.2s	remaining: 1h 14m 35s
2:	learn: 0.2644874	total: 1m 2s	remaining: 1h 8m 45s
3:	learn: 0.2383811	total: 1m 16s	remaining: 1h 2m 7s
4:	learn: 0.2223109	total: 1m 24s	remaining: 54m 56s
5:	learn: 0.2131348	total: 1m 32s	remaining: 49m 46s
6:	learn: 0.2053870	total: 1m 41s	remaining: 46m 26s
7:	learn: 0.1999146	total: 1m 48s	remaining: 43m 32s
8:	learn: 0.1942664	total: 1m 56s	remaining: 41m 15s
9:	learn: 0.1905505	total: 2m 4s	remaining: 39m 23s
10:	learn: 0.1870393	total: 2m 11s	remaining: 37m 47s
11:	learn: 0.1841779	total: 2m 19s	remaining: 36m 25s
12:	learn: 0.1816045	total: 2m 27s	remaining: 35m 17s
13:	learn: 0.1786934	total: 2m 39s	remaining: 35m 16s
14:	learn: 0.1761508	total: 3m 1s	remaining: 37m 14s
15:	learn: 0.1736454	total: 3m 18s	remaining: 38m 5s
16:	learn: 0.1714593	total: 3m 30s	remaining: 37m 41s
17:	learn: 0.1697909	total: 3m 55s	remaining: 39m 45s
18:	l

Модель CatBoost не достигла необходимого результата.

## Решающее дерево

In [54]:
#model_dtc = DecisionTreeClassifier()
#params_dtc = {'max_depth': [35]} #Сократил для ускорения работы, подбор проходил на более широком диапазоне.

#grid_dtc = GridSearchCV(model_dtc, params_dtc, cv=3)
#grid_dtc.fit(features_train, target_train)
#print(grid_dtc.best_params_)

In [57]:
model_dtc_0 = DecisionTreeClassifier(random_state=SEED, max_depth=35)
model_dtc_0.fit(features_train_0, target_train_0)
prediction_dtc_valid_0 = model_dtc_0.predict(features_valid_0)
show_me_quality_please(target_valid_0, prediction_dtc_valid_0)

Точность предсказаний: 0.947
-------------------------
F1-мера 0.689
-------------------------
Матрица ошибок:
 [[0.98958843 0.01041157]
 [0.42632067 0.57367933]]


Модель решающего дерева не достигла треуемого качества.

# 3. Анализ моделей

Требуемый порог F1-меры достигла модель логистической регрессии. Проверим модель на тестовой выборке.

In [58]:
prediction_lr_test_0 = model_lr.predict(features_test_0)
show_me_quality_please(target_test_0, prediction_lr_test_0)

Точность предсказаний: 0.95
-------------------------
F1-мера 0.765
-------------------------
Матрица ошибок:
 [[0.96568953 0.03431047]
 [0.19215323 0.80784677]]


Модель успешно ведёт себя на тестовой выборке.

# 4. Выводы

В ходе работы с текстами построили модели различных типов для классификации сообщений. Наибольшую эффективность показала модель логистической регрессии: F1-мера 0.765 на тестовой выборке. Модель CatBoost имеет второе качество, но она не достигла требуемого порога, также время и затраты на работу модели не позволяют утверждать о её эффективности.

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны